<a href="https://colab.research.google.com/github/uol-mediaprocessing-202021/medienverarbeitung-b-color-and-tone-correction/blob/develop/ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys

from zipfile import ZipFile

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
with ZipFile('/content/gdrive/MyDrive/huawei10000/huawei10000.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path="/content/gdrive/MyDrive/huawei10000/")

In [ ]:
with ZipFile('/content/gdrive/MyDrive/canon10000/canon10000.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path="/content/gdrive/MyDrive/canon10000/")

In [ ]:
with ZipFile('/content/gdrive/MyDrive/DPED-master/DPED-master.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path="/content/gdrive/MyDrive/DPED-master/")

FileNotFoundError: ignored

In [ ]:
with ZipFile('/content/gdrive/MyDrive/canon_test.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path="/content/gdrive/MyDrive/canon_test/")

In [ ]:
with ZipFile('/content/gdrive/MyDrive/huawei_test.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path="/content/gdrive/MyDrive/huawei_test/")

In [3]:
image = cv2.imread("/content/gdrive/MyDrive/huawei10000/huawei/0.jpg")

KeyboardInterrupt: ignored

In [ ]:
plt.imshow(image)

In [3]:
sys.path.append('/content/gdrive/MyDrive/DPED-master/DPED-master')

In [6]:
%cd /content/gdrive/MyDrive/DPED-master/DPED-master/

/content/gdrive/MyDrive/DPED-master/DPED-master


In [7]:
!python load_dataset.py model=iphone

In [8]:
!python train_model.py model=iphone train_size=10019

Traceback (most recent call last):
  File "train_model.py", line 155, in <module>
    feed_dict={phone_: phone_images, dslr_: dslr_images, adv_: all_zeros})
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 968, in run
    run_metadata_ptr)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1191, in _run
    feed_dict_tensor, options, run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1369, in _do_run
    run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1375, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1360, in _run_fn
    target_list, run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1453, in _call_tf_sessionrun
    run_metadata)
KeyboardInterrupt
